## 2.2　Newtonブースティング法の考え方

**abstract** : Newtonブースティング法は、教師あり学習が仮説空間に対する損失関数の最小化問題であることに注目し、数理最適化の手法であるNewton法を関数空間に拡張することで最適な仮説を導く方法を提案したものです。このことを理解するために、

A. Newton法の仕組み<br/>
B. Newtonブースティング法の仕組み

を説明します。

### A. Newton法の仕組み

**Newton法とそのalgorithm** : $d$個の実数からなるベクトルの全体$\mathbb{R}^d$を定義域に持つような関数$L(x)$の最適解の値を計算したいとします。要するに、\begin{eqnarray*}x^*=\mathrm{argmax}_{x}L(x)\end{eqnarray*}を考えているわけです。特に、関数$L(x)$が2階微分可能、要するに
* 1階導関数$\nabla_x L(x)$を持ち、この導関数に$x=a$を代入した値$\nabla_x L(a)$を計算できる。
* 2階導関数$\nabla^2_x L(x)$を持ち、この導関数に$x=a$を代入した値$\nabla^2_x L(a)$を計算できる。

このとき、以下のようなalgorithmが用いられることがあります。

**Input** : 
* $x_0$ : 初期値
* $\eta$ : 学習率
* $M$ : 繰り返しの最大数
* $\nabla_x L$ : 関数$L$の1階導関数
* $\nabla^2_x L$ : 関数$L$の2階導関数

**Process** :
1. $a\leftarrow x_0$　　# 解の初期値
2. for $i$ in $1,\cdots,M$:
3. 　　$a \leftarrow a – \eta \nabla^2_x L(a)^{-1}\nabla_x L(a)$
4. return $a$

これを**Newton法**といいます。

**Newton法のアイディア** : Newton法のalgorithmは、目的関数を2次近似することで現在の点から停留点に向かう方向を求め、その方向$\nabla^2_x L(a)^{-1}\nabla_x L(a)$に移動するという計算を繰り返すという流れを実装しています。このことを実際に1変数の目的関数の場合で確認してみましょう。

<center><img src="./imgs/newton.png" width=500px><br>Newton法の仕組み</center>

関数$L(x)$を現在の解$x=a$まわりで2次までTaylor展開することで、関数の2次近似$\tilde{L}(x)$を次のように得ます。\begin{eqnarray*}\tilde{L}(x)=L(a)+\nabla_xL(a)(x-a)+\frac{1}{2}\nabla^2_xL(a)(x-a)^2\end{eqnarray*}この式を微分すると\begin{eqnarray*}\nabla_x \tilde{L}(x)=\nabla_xL(a)+\nabla^2_xL(a)(x-a)\end{eqnarray*}が得られます。あとは、求めたい$\nabla_xL(x)=0$の代わりに$\nabla_x\tilde{L}(x)=0$の解$\hat{a}$を求めると、右辺から$\hat{a}=a-\nabla^2_x L(a)^{-1}\nabla_x L(a)$を得ることが出来ます。

### B. Newtonブースティング法の仕組み

Newtonブースティング法は、Newton法を実数ベクトルを入力にとる目的関数の最適化から、**仮説を入力にとる目的関数の最適化に拡張したもの**です。拡張の流れから説明するとやや難しいので、ここでは、Newtonブースティング法のalgorithmを天下りに説明して、紹介したalgorithmがなぜNewton法の拡張とみなせるのかという流れで説明します。

#### B1. Newtonブースティング法のalgorithm

**教師あり学習における最適化問題の復習** : 教師あり学習における最適化問題とは、事前にデータ$\mathcal{D}=\{(x_1,y_1),\cdots,(x_n,y_n)\}$を準備し、
1. 仮説空間$\mathcal{H}$
2. 損失関数$l(\hat{y_i},y)$

を定義したうえで、以下のような問題を解くことを言うのでした。\begin{eqnarray*}\hat{h}(x)&=&\mathrm{argmin}_{h\in\mathcal{H}}L(h)\\
\text{ where }L(h)&=&\sum_{i=1}^{n}l(h(x_i),y_i)\end{eqnarray*}

**Newtonブースティング法のalgorithm例** : 特に、仮説空間$\mathcal{H}$を決定木の仮説の和
\begin{eqnarray*}
h(x) &=& \sum_{i=0}^{M}h_{i}(x)
\end{eqnarray*}
の全体で定義します。また簡単のために、損失関数を2乗損失$l(\hat{y},y)=(\hat{y}-y)^2$にします。Newtonブースティング法では、教師あり学習における最適化問題に対して、この$h_{i}(x)$を以下のalgorithmに従って求めます。

**Input** :
* $\eta>0$ : 学習率
* $\nabla_h L$ : 目的関数$L$の1階導関数
* $\nabla^2_h L$ : 目的関数$L$の2階導関数
* $M$ : 繰り返しの最大回数

**Process** :
1. $\hat{h}(x)\leftarrow \bar{y}$
2. for $t$ in $1,\cdots,M$:
3. 　　残差$r_i=y_i-h(x_i)$, $i=1,\cdots,n$を計算する。
4. 　　$\mathcal{D}':=\{(x_1,-r_1),\cdots,(x_n,-r_n)\}$で$h_{t}(x)$を学習する。
5. 　　$\hat{h}(x)\leftarrow\hat{h}(x)+\eta h_{t}(x)$
6. return $\hat{h}(x)$

4行目で求めた$h_{t}(x)$によって、求める仮説$\hat{h}(x)$を$\eta h_t(x)$だけ更新することが、「仮説$h(x)$を入力にとる目的関数$L(h)$にNewton法を行う」ことに対応しています。

**[Remark]**  仮説空間を決定木の仮説の和の全体で定義することはalgorithmを導出のなかでは本質的ではありませんが、Newtonブースティング法の仮説空間としてこれを採用することが一般的なので、ここではそう設定しました。

#### B2. Newton法の拡張になっていることの説明（難）

**2次近似の導入** : 上記のalgorithmに対して、$t$回目の繰り返しで得られる仮説を$\hat{h}^{(t)}(x)$と書くことにします。目的関数$L(h)$は、関数$\hat{h}^{(t)}(x)$を中心に次のように近似することが出来ます。
\begin{eqnarray*}
\tilde{L}(h) &=& L(h^{(t)}) + \sum_{i=1}^{n} \left\{\frac{\partial l}{\partial\hat{y}}(h^{(t)}(x_i),y_i)(h(x_i)-h^{(t)}(x_i)) + \frac{1}{2}\frac{\partial^2 l}{\partial\hat{y}^2}(h^{(t)}(x_i),y_i)(h(x_i)-h^{(t)}(x_i))^2\right\}
\end{eqnarray*}
これを目的関数の2次近似と言います。

**2次近似をどう導出したか** : ここで、
\begin{eqnarray*}
\langle F, G\rangle &:=& \sum_{i=1}^nF(x_i)G(x_i)
\end{eqnarray*}
と定義します。また、現在の仮説を$h^{(t)}(x)$と書くことにします。目的関数$L(h)$の2次近似をTaylor展開のアナロジーで考えて、

\begin{eqnarray*}
\tilde{L}(h) &:=& L(\hat{h}^{(t)}) + \langle\nabla_hL(\hat{h}^{(t)}), h-\hat{h}^{(t)}\rangle + \frac{1}{2} \langle\nabla^2_hL(\hat{h}^{(t)}), (h-\hat{h}^{(t)})^2\rangle
\end{eqnarray*}

と表現することにしましょう。このとき、1階導関数・2階導関数はそれぞれ

\begin{eqnarray*}
\nabla_{h}L(\hat{h}^{(t)})(a) &=& \begin{cases}\frac{\partial l}{\partial\hat{y}}(\hat{h}^{(t)}(x_i),y_i)\text{ if } a=x_i\\0\text{ otherwise}\end{cases}\\
\nabla^2_{h}L(\hat{h}^{(t)})(a) &=& \begin{cases}\frac{\partial^2 l}{\partial\hat{y}^2}(\hat{h}^{(t)}(x_i),y_i)\text{ if }a=x_i\\0\text{ otherwise}\end{cases}
\end{eqnarray*}

なので、

\begin{eqnarray*}
\tilde{L}(h) &=& L(\hat{h}^{(t)}) + \sum_{i=1}^{n} \left\{\frac{\partial l}{\partial\hat{y}}(\hat{h}^{(t)}(x_i),y_i)(h(x_i)-\hat{h}^{(t)}(x_i)) + \frac{1}{2}\frac{\partial^2 l}{\partial\hat{y}^2}(\hat{h}^{(t)}(x_i),y_i)(h(x_i)-\hat{h}^{(t)}(x_i))^2\right\}
\end{eqnarray*}

が従います。

**algorithmの導出** : 上のalgorithmにおける仮説の更新分$h_t(x)$は、2次近似の$h(x)-\hat{h}^{(t)}(x)$に対応しています。要するに$h(x):=h(x)-\hat{h}^{(t)}(x)$です。また、損失関数が2乗損失$l(\hat{y},y)=(\hat{y}-y)^2$ならば、

\begin{eqnarray*}
\nabla_{h}L(\hat{h}^{(t)})(a) &=& \begin{cases}2(\hat{h}^{(t)}-y)\text{ if } a=x_i\\0\text{ otherwise}\end{cases}\\
\nabla^2_{h}L(\hat{h}^{(t)})(a) &=& \begin{cases}2\text{ if }a=x_i\\0\text{ otherwise}\end{cases}
\end{eqnarray*}


が従います。これから、目的関数の2次近似$\tilde{L}(h)$を次のように書き換えられるわけです。

\begin{eqnarray*}
\tilde{L}(h) &=& L(\hat{h}^{(t)}(h)) + 2\sum_{i=1}^{n}(\hat{h}^{(t)}(x_i)-y_i)h_{t}(x_i) + \sum_{i=1}^{n}h_{t}(x_i)^2\\
&=& \sum_{i=1}^{n}\left\{h_{t}(x_i)-(y_i-\hat{h}^{(t)}(x_i))\right\}^2 + \mathrm{const.}
\end{eqnarray*}

これより2次近似が最も小さくなるのは、$h_{t}(x_i)$を$\mathcal{D}':=\{(x_1,y_1-\hat{h}^{(t)}(x_1)),\cdots,(x_n,y_n-\hat{h}^{t)}(x_n))\}$で学習させたときです。この学習を決定木で行えば、上記のalgorithmになります。